In [1]:
from data_wrangling_components.pipeline import DefaultPipeline
import data_wrangling_components.types as types

## Aggregated Lookup



### Load files into the table store

In [2]:
aggregated_lookup = DefaultPipeline()

aggregated_lookup.add_dataset('companies', path='../data/companies.csv')
aggregated_lookup.add_dataset('products', path='../data/products.csv')

### Create steps

In [3]:
aggregated_lookup.add(
    types.Step(
        types.Verb.Join,
        "companies",
        "join-1",
        args={"other": "products", "on": ["ID"]},
    )
)

aggregated_lookup.add(
    types.Step(
        types.Verb.Filter,
        "join-1",
        "filter-1",
        args={
            "to": "",
            "column": "FY21 Sales",
            "operator": types.NumericComparisonOperator.Gte,
            "type": types.FilterCompareType.Value,
            "value": 10000,
        },
    )
)

aggregated_lookup.add(
    types.Step(
        types.Verb.Aggregate,
        "filter-1",
        "aggregate-1",
        args={
            "to": "Total sales >= 10k",
            "groupby": "ID",
            "column": "FY21 Sales",
            "operation": types.FieldAggregateOperation.Sum,
        },
    )
)

aggregated_lookup.add(
    types.Step(
        types.Verb.Lookup,
        "companies",
        "lookup-1",
        args={"other": "aggregate-1", "on": ["ID"], "columns": ["Total sales >= 10k"]},
    )
)

### Run pipeline

In [4]:
aggregated_lookup.run()

TableContainer(id='lookup-1', name='lookup-1', table=   ID       Name  Employees     US  Total sales >= 10k
0   1  Microsoft     160000   True             3206000
1   2      Apple     150000   True             4180000
2   3     Google     135000   True              660000
3   4     Amazon    1250000   True               38000
4   5    Samsung     270000  False              630000, metadata=None, context=None)

In [5]:
aggregated_lookup._store.get('filter-1')

LazyTableStorage(container=TableContainer(id='filter-1', name='filter-1', table=    ID       Name  Employees     US         Product  FY20 Sales  FY21 Sales
0    1  Microsoft     160000   True   Xbox Series X      9000.0       20000
1    1  Microsoft     160000   True     Surface Pro     80000.0       85000
2    1  Microsoft     160000   True    Surface Book      1300.0       11000
3    1  Microsoft     160000   True         Windows   4000000.0     3000000
4    1  Microsoft     160000   True           Azure     50000.0       90000
5    2      Apple     150000   True       iPhone 13     90000.0      120000
6    2      Apple     150000   True  MacBook Pro M1     89000.0       60000
7    2      Apple     150000   True             iOS   5000000.0     4000000
8    3     Google     135000   True           MacOS     30000.0       25000
9    3     Google     135000   True         Android    800000.0      600000
10   3     Google     135000   True     Google Maps     39000.0       35000
11   4  

### Check Results

In [6]:
# List all results in the store:
print(aggregated_lookup.list_store())

# Check final result:
lookup_final_result = aggregated_lookup.get_dataset('lookup-1')

## You can save the output to a csv file with:
# lookup_final_result.to_csv('name.csv')

## You can check other intermediate results by using the store and providing
## any name from the list above, for example: 
# aggregated_lookup_store.get('filter-1')

# Preview table:
lookup_final_result.head()

['companies', 'products', 'join-1', 'filter-1', 'aggregate-1', 'lookup-1']


,ID,Name,Employees,US,Total sales >= 10k
0,1,Microsoft,160000,True,3206000
1,2,Apple,150000,True,4180000
2,3,Google,135000,True,660000
3,4,Amazon,1250000,True,38000
4,5,Samsung,270000,False,630000


## Reading from json spec

In [7]:
import json

with open('binning.json', 'r') as binning_spec:
    pipeline_spec = json.loads(binning_spec.read())

In [8]:

# Create pipeline
binning_pipeline = DefaultPipeline.from_json(pipeline_spec['steps'])

# Load inputs into store
binning_pipeline.add_dataset('data/products.csv', path='../data/products.csv')

In [9]:
# Run pipeline
binning_pipeline.run()


TableContainer(id='bin-5', name='bin-5', table=    ID         Product  FY20 Sales  FY21 Sales   Binned
0    1        Xbox One     10000.0        9000    20000
1    1   Xbox Series X      9000.0       20000    20000
2    1     Surface Pro     80000.0       85000    20000
3    1    Surface Book      1300.0       11000    20000
4    1         Windows   4000000.0     3000000  1000000
5    1           Azure     50000.0       90000    20000
6    2       iPhone 13     90000.0      120000   120000
7    2  Watch Series 7      2000.0        3000    20000
8    2  MacBook Pro M1     89000.0       60000    20000
9    2             iOS   5000000.0     4000000  1000000
10   3           MacOS     30000.0       25000    20000
11   3         Android    800000.0      600000   520000
12   3          Stadia         NaN        8000    20000
13   3     Google Maps     39000.0       35000    20000
14   4           Gmail     38000.0       38000    20000
15   5          Galaxy    700000.0      600000   520000
1

In [10]:
# Preview result in store
binning_pipeline.get_dataset('bin-5').head()

,ID,Product,FY20 Sales,FY21 Sales,Binned
0,1,Xbox One,10000.0,9000,20000
1,1,Xbox Series X,9000.0,20000,20000
2,1,Surface Pro,80000.0,85000,20000
3,1,Surface Book,1300.0,11000,20000
4,1,Windows,4000000.0,3000000,1000000


## Another Example

In [11]:
import json

with open('compound-filter-aggregate.json', 'r') as compound_spec:
    pipeline_spec = json.loads(compound_spec.read())

In [12]:

# Create pipeline
compound_pipeline = DefaultPipeline.from_json(pipeline_spec['steps'])


# Load inputs into store
compound_pipeline.add_dataset('data/companies.csv', path='../data/companies.csv')
compound_pipeline.add_dataset('data/products.csv', path='../data/products.csv')

In [13]:
compound_pipeline.run()
print(compound_pipeline.list_store())
compound_pipeline.get_dataset('filter-aggregate-lookup-final-output-table').head()

['data/companies.csv', 'data/products.csv', 'filter-aggregate-lookup-final-output-table']


,ID,Name,Employees,US,Growth
0,1,Microsoft,160000,True,4.0
1,2,Apple,150000,True,2.0
2,3,Google,135000,True,NaN
3,4,Amazon,1250000,True,NaN
4,5,Samsung,270000,False,2.0


## Save pipeline Spec to Json

Note: When saving a pipeline, you save the specification and not the store, so you need to create and load the inputs into the store and run the pipeline again to recreate the results

In [14]:
aggregated_lookup.to_json('aggregate-lookup-save-test.json')

In [15]:
import json

with open('aggregate-lookup-save-test.json', 'r') as spec:
    pipeline_spec = json.loads(spec.read())


# Create pipeline
pipeline = DefaultPipeline.from_json(pipeline_spec['steps'])

pipeline.add_dataset('companies', path='../data/companies.csv')
pipeline.add_dataset('products', path='../data/products.csv')

# Run pipeline
pipeline.run()

TableContainer(id='lookup-1', name='lookup-1', table=   ID       Name  Employees     US  Total sales >= 10k
0   1  Microsoft     160000   True             3206000
1   2      Apple     150000   True             4180000
2   3     Google     135000   True              660000
3   4     Amazon    1250000   True               38000
4   5    Samsung     270000  False              630000, metadata=None, context=None)

In [16]:
lookup_1 = pipeline.get_dataset('lookup-1')
lookup_1

,ID,Name,Employees,US,Total sales >= 10k
0,1,Microsoft,160000,True,3206000
1,2,Apple,150000,True,4180000
2,3,Google,135000,True,660000
3,4,Amazon,1250000,True,38000
4,5,Samsung,270000,False,630000


In [17]:
# Test result is the same from the first execution (original pipeline creation)
from pandas.testing import assert_frame_equal

assert_frame_equal(lookup_1, lookup_final_result)

## Save store to a zip file

In [18]:
pipeline.save_store('./test.zip')

## Read store from zip file

In [19]:
new_pipeline = DefaultPipeline()
new_pipeline.load_store('./test.zip')
new_pipeline.list_store()

['companies', 'products', 'join-1', 'filter-1', 'aggregate-1', 'lookup-1']

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame({
    'A': [True, False, False, True],
    'B': [True, True, False, True],
    'C': [False, False, False, True]
})

In [4]:
df

,A,B,C
0,True,True,False
1,False,True,False
2,False,False,False
3,True,True,True


In [5]:
df.any(axis='columns')

0     True
1     True
2    False
3     True
dtype: bool

In [6]:
df.all(axis='columns')

0    False
1    False
2    False
3     True
dtype: bool

In [7]:
~df.any(axis='columns')

0    False
1    False
2     True
3    False
dtype: bool

In [8]:
import numpy as np
from functools import reduce

True ^ False ^ False

True

In [9]:
True ^ True ^ True

True

In [10]:
df.any(axis=1)

0     True
1     True
2    False
3     True
dtype: bool

In [11]:
df.sum(axis=1).apply(lambda x: 0 < x < 3)

0     True
1     True
2    False
3    False
dtype: bool

In [12]:
df.sum(axis=1).apply(lambda x: x == 1)

0    False
1     True
2    False
3    False
dtype: bool

In [13]:
df.astype({'A': int})

,A,B,C
0,1,True,False
1,0,True,False
2,0,False,False
3,1,True,True


In [14]:
df

,A,B,C
0,True,True,False
1,False,True,False
2,False,False,False
3,True,True,True


In [15]:
import pandas as pd

In [205]:
df = pd.DataFrame({
    'key': ['id', 'sales', 'id', 'sales', 'id', 'sales', 'sales', 'id', 'sales', 'sales'],
    'value': [1, 100, 1, 200, 2, 150, 300, 3, 12, 31],
    'value2': [1, 1, 1, 2, 2, 1.5, 3, 3, 1.2, 3.1],
})

In [63]:
df

,key,value,value2
0,id,1,1.0
1,sales,100,1.0
2,id,1,1.0
3,sales,200,2.0
4,id,2,2.0
5,sales,150,1.5
6,sales,300,3.0
7,id,3,3.0
8,sales,12,1.2
9,sales,31,3.1


In [64]:
df.pivot_table(values=['value', 'value2'], columns='key', aggfunc=sum)

key,id,sales
value,7.0,793.0
value2,7.0,11.8


In [58]:
df_unfold = pd.DataFrame({
    'key': ['id', 'name', 'id', 'name', 'id', 'name'],
    'value': [1, 'Bob', 2, 'Joe', 3, 'Jenny']
})



df_unfold

,key,value
0,id,1
0,name,Bob
1,id,2
1,name,Joe
2,id,3
2,name,Jenny


In [194]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'id': [1,1,2,2,2],
    'value': [10,15,1,11,18],
})

In [195]:
df

,id,value
0,1,1
1,1,2
2,2,3
3,2,4
4,2,5


In [196]:
# Row number
df_row_number = df.copy()
df_row_number['output'] = np.arange(1,len(df)+1)
df_row_number

,id,value,output
0,1,1,1
1,1,2,2
2,2,3,3
3,2,4,4
4,2,5,5


In [197]:
a = df.groupby('id').apply(lambda x: pd.DataFrame({'row_number': np.arange(1, len(x)+1)}))

In [198]:
a.reset_index()['row_number']

0    1
1    2
2    1
3    2
4    3
Name: row_number, dtype: int32

In [204]:
df

,id,value
0,1,1
1,1,2
2,2,3
3,2,4
4,2,5


In [223]:
# row number
a = df.groupby('ID')

a.keys

'ID'

In [224]:
window = a['FY21 Sales'].rolling(window=len(df), min_periods=1).count()
window

ID    
1   0     1.0
    1     2.0
    2     3.0
    3     4.0
    4     5.0
    5     6.0
2   6     1.0
    7     2.0
    8     3.0
    9     4.0
3   10    1.0
    11    2.0
    12    3.0
    13    4.0
4   14    1.0
5   15    1.0
    16    2.0
    17    3.0
    18    4.0
Name: FY21 Sales, dtype: float64

In [227]:
b = a.obj
b['result'] = window.reset_index()['FY21 Sales']

In [230]:
c = b.groupby(a.keys)

c

In [193]:
# rank
df.groupby('id').rolling(window=len(df),min_periods=1).count()

KeyError: 'id'

In [190]:
(df['FY21 Sales'].rolling(window=len(df),min_periods=1).count() - 1) / (len(df) - 1)

0     0.000000
1     0.055556
2     0.111111
3     0.166667
4     0.222222
5     0.277778
6     0.333333
7     0.388889
8     0.444444
9     0.500000
10    0.555556
11    0.611111
12    0.666667
13    0.722222
14    0.777778
15    0.833333
16    0.888889
17    0.944444
18    1.000000
Name: FY21 Sales, dtype: float64

In [191]:
# cume_dist
df['FY21 Sales'].rolling(window=len(df),min_periods=1).count() / len(df)

0     0.052632
1     0.105263
2     0.157895
3     0.210526
4     0.263158
5     0.315789
6     0.368421
7     0.421053
8     0.473684
9     0.526316
10    0.578947
11    0.631579
12    0.684211
13    0.736842
14    0.789474
15    0.842105
16    0.894737
17    0.947368
18    1.000000
Name: FY21 Sales, dtype: float64

In [231]:
df['FY21 Sales'].rolling(window=len(df),min_periods=1).rank(pct=True)

0     1.000000
1     1.000000
2     1.000000
3     0.500000
4     1.000000
5     0.833333
6     0.857143
7     0.125000
8     0.555556
9     1.000000
10    0.454545
11    0.833333
12    0.153846
13    0.500000
14    0.533333
15    0.843750
16    0.058824
17    0.111111
18    0.473684
Name: FY21 Sales, dtype: float64

In [45]:
# first value
df.groupby('id').rolling(window=len(df), min_periods=1).apply(lambda x: x.iloc[0])

value
id         
1  0   10.0
   1   10.0
2  2    1.0
   3    1.0
   4    1.0

In [46]:
# last value
df.groupby('id').rolling(window=pd.api.indexers.FixedForwardWindowIndexer(window_size=len(df)), min_periods=1).apply(lambda x: x.iloc[-1])

value
id         
1  0   15.0
   1   15.0
2  2   18.0
   3   18.0
   4   18.0

In [65]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'id': [1,1,2,2,2],
    'value': [10,np.nan,np.nan,2,18],
})

In [66]:
# fill_up

df.groupby('id').rolling(window=pd.api.indexers.FixedForwardWindowIndexer(window_size=len(df)), min_periods=1).apply(lambda x: x.dropna().iloc[0] if np.isnan(x.iloc[0]) else x.iloc[0])

value
id         
1  0   10.0
   1    NaN
2  2    2.0
   3    2.0
   4   18.0

In [72]:
# fill_down

df.groupby('id').rolling(window=len(df), min_periods=1).apply(lambda x: x.dropna().iloc[-1])

value
id         
1  0   10.0
   1   10.0
2  2    NaN
   3    2.0
   4   18.0

In [183]:
df.dtypes

id      int64
bool     bool
dtype: object

In [207]:
df_path = "C:/Users/andresmor/Projects/github/data-wrangling-components/javascript/webapp/public/data/products.csv"

df = pd.read_csv(df_path, engine="python", on_bad_lines=lambda x: x)

df

,ID,Product,FY20 Sales,FY21 Sales
0,1,Xbox One,10000.0,9000
1,1,Xbox Series X,9000.0,20000
2,1,Surface Pro,80000.0,85000
3,1,Surface Book,1300.0,11000
4,1,Windows,4000000.0,3000000
5,1,Azure,50000.0,90000
6,2,iPhone 13,90000.0,120000
7,2,Watch Series 7,2000.0,3000
8,2,MacBook Pro M1,89000.0,60000
9,2,iOS,5000000.0,4000000


In [186]:
df['FY21 Sales'].rolling(window=len(df),min_periods=1).count()

0      1.0
1      2.0
2      3.0
3      4.0
4      5.0
5      6.0
6      7.0
7      8.0
8      9.0
9     10.0
10    11.0
11    12.0
12    13.0
13    14.0
14    15.0
15    16.0
16    17.0
17    18.0
18    19.0
Name: FY21 Sales, dtype: float64

In [180]:
df['FY21 Sales'].expanding(min_periods=1).rank()

0      1.0
1      2.0
2      3.0
3      2.0
4      5.0
5      5.0
6      6.0
7      1.0
8      5.0
9     10.0
10     5.0
11    10.0
12     2.0
13     7.0
14     8.0
15    13.5
16     1.0
17     2.0
18     9.0
Name: FY21 Sales, dtype: float64

In [166]:
df

,ID,Product,FY20 Sales,FY21 Sales,rank,cume_dist
0,1,Xbox One,10000.0,9000,5.0,0.263158
1,1,Xbox Series X,9000.0,20000,7.0,0.368421
2,1,Surface Pro,80000.0,85000,13.0,0.684211
3,1,Surface Book,1300.0,11000,6.0,0.315789
4,1,Windows,4000000.0,3000000,18.0,0.947368
5,1,Azure,50000.0,90000,14.0,0.736842
6,2,iPhone 13,90000.0,120000,15.0,0.789474
7,2,Watch Series 7,2000.0,3000,3.0,0.157895
8,2,MacBook Pro M1,89000.0,60000,12.0,0.631579
9,2,iOS,5000000.0,4000000,19.0,1.000000


In [184]:
df = pd.DataFrame({
    'A': [1,1,1,1,1,1,1,1,1,1]
})


df['A']

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
Name: A, dtype: float64

In [216]:
df.groupby('ID')['FY21 Sales'].

In [232]:
c

In [233]:
c.sample()

,ID,Product,FY20 Sales,FY21 Sales,result
2,1,Surface Pro,80000.0,85000,3.0
6,2,iPhone 13,90000.0,120000,1.0
10,3,MacOS,30000.0,25000,1.0
14,4,Gmail,38000.0,38000,1.0
16,5,Galaxy Note,3.0,1,2.0


In [234]:
c.obj

,ID,Product,FY20 Sales,FY21 Sales,result
0,1,Xbox One,10000.0,9000,1.0
1,1,Xbox Series X,9000.0,20000,2.0
2,1,Surface Pro,80000.0,85000,3.0
3,1,Surface Book,1300.0,11000,4.0
4,1,Windows,4000000.0,3000000,5.0
5,1,Azure,50000.0,90000,6.0
6,2,iPhone 13,90000.0,120000,1.0
7,2,Watch Series 7,2000.0,3000,2.0
8,2,MacBook Pro M1,89000.0,60000,3.0
9,2,iOS,5000000.0,4000000,4.0
